# WizardsCo: A Data Wizards Assignment

### Create DataBase and connect to it

In [2]:
# Import required modules
import sqlite3
import pandas as pd

# Connecting to the WizardsCo database
connection = sqlite3.connect('wizards_co.db')

# Creating a cursor object to execute
# SQL queries on a database table
cursor = connection.cursor()

### Create different SQL tables from CSV files using pandas library

In [67]:
customer = pd.read_csv('files/customer.csv')
customer.to_sql('customer', connection)

In [68]:
country = pd.read_csv('files/country.csv')
country.to_sql('country', connection)

In [69]:
installation = pd.read_csv('files/installation.csv')
installation.to_sql('installation', connection)

In [70]:
product_category = pd.read_csv('files/product_category.csv')
product_category.to_sql('product_category', connection)

In [71]:
product = pd.read_csv('files/product.csv')
product.to_sql('product', connection)

### Create new empty table "fact_installation"

In [72]:
create_table = """CREATE TABLE fact_installation(
                installation_id INTEGER,
                product_id INTEGER,
                customer_id INTEGER,
                product_category TEXT,
                price TEXT,
                region TEXT,
                installation_date TEXT,
                premium_customer TEXT);"""

cursor.execute(create_table)

### Import relevant data to the newly created table

In [74]:
import_data = """INSERT INTO fact_installation(installation_id, product_id, customer_id, product_category, price, region, installation_date, premium_customer)
                SELECT i.id, p.id, cu.id, pc.name, p.price, co.region, i.installation_date, cu.premium_customer
                FROM installation AS i
                INNER JOIN product AS p ON p.id = i.product_id
                INNER JOIN customer AS cu ON cu.id = i.customer_id
                INNER JOIN product_category AS pc ON pc.id = p.category_id
                INNER JOIN country AS co ON co.id = cu.country_id"""

cursor.execute(import_data)
connection.commit()

### Change installation_date column to installation_month

In [76]:
#first change the date by selecting only the month from yyyy-mm-dd format
update_date = """UPDATE fact_installation
                SET installation_date = SUBSTR(installation_date, 6, 2)"""

cursor.execute(update_date)

In [77]:
#then change the column name to installation_month
update_name = """ALTER TABLE fact_installation
                RENAME COLUMN installation_date TO installation_month"""

cursor.execute(update_name)

In [3]:
#read table to see results
pd.read_sql_query('SELECT * FROM fact_installation', connection)

,installation_id,product_id,customer_id,product_category,price,region,installation_month,premium_customer
0,1000,1005,1004,IT & Network Stuff,789€,Europe,10,no
1,1001,1003,1005,Beauty Accessories,90€,Europe,10,yes
2,1002,1000,1000,Dangerous Items,12345€,Europe,09,yes
3,1003,1000,1001,Dangerous Items,12345€,Asia,09,yes
4,1004,1002,1002,Miscellaneous,12€,America,08,no
5,1005,1005,1003,IT & Network Stuff,789€,Europe,08,yes
6,1006,1005,1004,IT & Network Stuff,789€,Europe,07,no
7,1007,1004,1001,Beauty Accessories,5€,Asia,07,yes
8,1008,1003,1000,Beauty Accessories,90€,Europe,10,yes
9,1009,1004,1004,Beauty Accessories,5€,Europe,05,no


In [79]:
#close the connection
cursor.close()